# Linéarisation et équilibrage d'un ABR

In [2]:
class Node:
    def __init__(self,val):
        self.val = val
        self.droite = None
        self.gauche = None

In [4]:
def linearisation(racine):
    if racine == None:
        return None, None, 0
    tete, queue, cnt = linearisation(racine.gauche)
    
    if cnt == 0:
        tete = queue = racine
    else:
        queue.droite = racine
        queue = queue.droite
    cnt += 1
    
    t2,q2,cnt2 = linearisation(racine.droite)
    queue.droite = t2
    queue = q2
    cnt += cnt2
    
    return tete, queue, cnt